<a href="https://colab.research.google.com/github/ryderwishart/biblical-machine-learning/blob/main/levinsohn_discourse_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook queries a `macula-greek` API endpoint to gather all of the features available from Levinsohn's discourse features, and then it queries a specific label in a specified passage in order to return all of the instances of that feature from the passage.

In [1]:
import requests
import json

In [4]:
ENDPOINT = 'https://macula-atlas-api-qa-25c5xl4maa-uk.a.run.app/graphql/'
headers = {"Content-Type": "application/json"}

# Query API for all of the discourse-feature types

In [5]:
annotation_types_response = requests.post(ENDPOINT, data=json.dumps(
    {
    "query": """
        query {
            annotationFeatures {
                data
            }
        }
    """,
}
), headers=headers)

if annotation_types_response.status_code == 200:
    types_raw = annotation_types_response.json()
    annotation_types_response_size_kb = len(annotation_types_response.content) / 1024
    print(f"annotation_types_response size: {annotation_types_response_size_kb:.2f} KB")
else:
    print(f"Error: {annotation_types_response.status_code} - {annotation_types_response.text}")

annotation_types_response size: 21.15 KB


In [6]:
# Inspect one of the discourse feature types data
types_raw['data']['annotationFeatures'][1]

{'data': {'name': 'Historical Perfect',
  'type': {'name': 'markup'},
  'license': {'url': 'https://github.com/biblicalhumanities/levinsohn/blob/master/LICENSE.md'},
  'product': {'text': 'Levinsohn Greek New Testament Discourse Features',
   'version': '1.0'},
  'copyright': '©2016 SIL International',
  'description': 'Highlights not the speech or act to which it refers but the event(s) that follow (DFNTG §12.2).',
  'defaultstyle': {'label': '',
   'bkcolor': '',
   'brdrpos': 'A,U,B,E',
   'newline': '',
   'brdrline': 'Solid',
   'brdrcolor': '#FF00FF00',
   'fontcolor': '',
   'fontstyle': 'None',
   'indentlevel': '',
   'newlineafter': '',
   'indentlevelafter': ''}}}

In [7]:
# Extract just the most useful information into a `types` dictionary 
types = {}
for t in types_raw['data']['annotationFeatures']:
    if t['data']:
        types[t['data']['name']] = {
            'description': t['data']['description'],
            }

# Print out all the discourse features types with their descriptions
for t in types:
    print(t + ':\t', types[t])

Historical Perfect:	 {'description': 'Highlights not the speech or act to which it refers but the event(s) that follow (DFNTG §12.2).'}
Specific Circumstance:	 {'description': 'The function of ἐγενετο ‘it came about’ and an immediately following temporal expression varies with the author (see DFNTG §10.3). In Matthew’s Gospel, it usually marks major divisions in the book (e.g. Mt 7:28). In Luke-Acts, in contrast, ‘it picks out from the general background the specific circumstance for the foreground events that are to follow’ (ibid.), as in Acts 9:37 (see also Mt 9:10).'}
Verb Focus+:	 {'description': 'Verb in final position in clause demonstrates verb focus.'}
Articular Pronoun:	 {'description': 'Articular pronoun, which often introduces an ‘intermediate step’ in a reported conversation.'}
Topical Genitive:	 {'description': 'A genitival constituent that is nominal is preposed within the noun phrase for two purposes: 1) to bring it into focus; 2) within a point of departure, to indicate

# Query the API for a specific discourse feature by label in a given book or passage

In [13]:
FEATURE = 'OT quotes'

In [15]:
PASSAGE = '2CO'

In [25]:
query = """
query AnnotationFeatures($filters1: AnnotationFeatureFilter, $filters2: AnnotationFilter) {
  annotationFeatures(filters: $filters1) {
    label
    uri
    instances(filters: $filters2) {
      uri
      tokens {
        ref
        wordValue
        xmlId
        lemma
        data
      }
    }
  }
}
"""

variables = {
  "filters1": {
    "reference": PASSAGE
  },
  "filters2": {
    "reference": PASSAGE,
    "featureLabel": FEATURE
  }
}

payload = {
    "query": query,
    "variables": variables,
}

In [26]:
response = requests.post(ENDPOINT, data=json.dumps(payload), headers=headers)

if response.status_code == 200:
    data = response.json()
    # print(json.dumps(data, indent=2))
    response_size_kb = len(response.content) / 1024
    print(f"Response size: {response_size_kb:.2f} KB")
else:
    print(f"Error: {response.status_code} - {response.text}")


Response size: 56.95 KB


In [28]:
# Filter out empty feature sets from results
results = [feature for feature in data['data']['annotationFeatures'] if len(feature['instances']) > 0]

In [29]:
for feature in results:
    # print(feature['uri'])
    
    # Print any non-zero results
    if len(feature['instances']) > 0:
        print(feature['label'], '- Number of instances: ', len(feature['instances']))

OT quotes - Number of instances:  21


In [31]:
# Print out the first 5 instances of text
for instance in results[0]['instances']:
    tokens = instance['tokens']
    first_ref = tokens[0]['ref']
    last_ref = tokens[-1]['ref']
    text = ' '.join([token['wordValue'] for token in tokens])
    glosses = ' '.join([token['data']['gloss'] for token in tokens])
    print(first_ref, '-', last_ref)
    print(text)
    print(glosses + '\n')

2CO 4:13!11 - 2CO 4:13!13
Ἐπίστευσα διὸ ἐλάλησα
I believed therefore I have spoken

2CO 6:2!3 - 2CO 6:2!6
Καιρῷ δεκτῷ ἐπήκουσά σου
In a time acceptable I listened to you

2CO 6:2!7 - 2CO 6:2!12
καὶ ἐν ἡμέρᾳ σωτηρίας ἐβοήθησά σοι
and in a day of salvation I helped you

2CO 6:16!19 - 2CO 6:16!23
Ἐνοικήσω ἐν αὐτοῖς καὶ ἐνπεριπατήσω
I will dwell in them and will walk among [them]

2CO 6:16!24 - 2CO 6:16!27
καὶ ἔσομαι αὐτῶν Θεός
and I will be their God

2CO 6:16!28 - 2CO 6:16!32
καὶ αὐτοὶ ἔσονταί μου λαός
and they will be to Me a people

2CO 6:17!2 - 2CO 6:17!5
ἐξέλθατε ἐκ μέσου αὐτῶν
come out from the midst of them

2CO 6:17!6 - 2CO 6:17!7
καὶ ἀφορίσθητε
and be separate

2CO 6:17!10 - 2CO 6:17!13
καὶ ἀκαθάρτου μὴ ἅπτεσθε
and unclean nothing touch

2CO 6:17!14 - 2CO 6:17!16
κἀγὼ εἰσδέξομαι ὑμᾶς
and I will receive you

2CO 6:18!2 - 2CO 6:18!2
ἔσομαι
I will be

2CO 6:18!4 - 2CO 6:18!5
εἰς Πατέρα
for a father

2CO 6:18!6 - 2CO 6:18!6
καὶ
and

2CO 6:18!9 - 2CO 6:18!11
μοι εἰς υἱοὺς
to Me for so